#  两层FC层做分类：MNIST

在本教程中，我们来实现一个非常简单的两层全连接层来完成MNIST数据的分类问题。

输入[-1,28*28], FC1 有 1024 个neurons， FC2 有 10 个neurons。这么简单的一个全连接网络，结果测试准确率达到了 0.98。还是非常棒的！！！


- get_variable()和variable()混用会自动处理命名问题
- variable_scope() 会在变量名前加入scope的命名空间

tf是根据变量名来区分是否是同一变量，需要使用reuse=tf.AUTO_REUSE进行制定变量


In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf

In [2]:
#导入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('../data/MNIST_data',one_hot=False, source_url='http://yann.lecun.com/exdb/mnist/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print('train_data image shape :',mnist.train.images.shape)
print('train_data label shape :',mnist.train.labels.shape)

train_data image shape : (55000, 784)
train_data label shape : (55000,)


In [15]:
def get_var_scope(scope_name,shape) :
    with tf.variable_scope(scope_name,reuse=tf.AUTO_REUSE) as scope:
#         scope.reuse_variables()
        try:
            w=tf.get_variable('w',shape,initializer=tf.truncated_normal_initializer(stddev=0.1))
            b=tf.get_variable('b',[shape[1]],initializer=tf.constant_initializer(0.1))
        except ValueError as e:
            w=tf.get_variable('w',shape)
            b=tf.get_variable('b',[shape[1]])
        return w,b
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.int64,[None])

W_fc1,b_fc1=get_var_scope('fc5',[784,1024])
h_fc1=tf.nn.relu(tf.matmul(x,W_fc1)+b_fc1)

W_fc2,b_fc2=get_var_scope('fc6',[1024,10])
y_hat=(tf.matmul(h_fc1,W_fc2)+b_fc2)

cost=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat,labels=y))


with tf.variable_scope('optimizer',reuse=tf.AUTO_REUSE):
    optimizer=tf.train.AdamOptimizer(1e-3).minimize(cost)

correct_prediction=tf.equal(tf.argmax(y_hat,1),y)
acc=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        x_batch,y_batch=mnist.train.next_batch(batch_size=100)
        _cost,_acc,_=sess.run([cost,acc,optimizer],feed_dict={x:x_batch,y:y_batch})
        if not i%500:
            test_cost,test_acc=sess.run([cost,acc],feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print('step {},train_cost={:.6f},acc={:.6f};test_cost:{:.6f},test_acc:{:.6f}'.format(i,_cost,_acc,test_cost,test_acc))

step 0,train_cost=221.197067,acc=0.130000;test_cost:209.098679,test_acc:0.082000
step 500,train_cost=10.145953,acc=0.870000;test_cost:7.630925,test_acc:0.882900
step 1000,train_cost=6.117918,acc=0.890000;test_cost:4.997635,test_acc:0.914000
step 1500,train_cost=2.645495,acc=0.940000;test_cost:4.009112,test_acc:0.924800
step 2000,train_cost=4.094882,acc=0.890000;test_cost:3.416116,test_acc:0.932000
step 2500,train_cost=2.471544,acc=0.930000;test_cost:2.944244,test_acc:0.938800
step 3000,train_cost=0.217323,acc=0.970000;test_cost:2.813977,test_acc:0.941500
step 3500,train_cost=0.334009,acc=0.970000;test_cost:2.585214,test_acc:0.944800
step 4000,train_cost=1.908447,acc=0.930000;test_cost:2.408782,test_acc:0.945100
step 4500,train_cost=0.465369,acc=0.980000;test_cost:2.314977,test_acc:0.947600
